In [1]:
import pandas as pd
import os
import csv
import glob
import numpy as np
import collections
import matplotlib.pyplot as plt

Prices = collections.namedtuple('Prices', field_names=['open', 'high', 'low', 'close', 'volume'])

In [2]:
train_path = "D:\Practical_Reinforcement_Learning\chapter9\data\ch08-small-quotes\YNDX_150101_151231.csv"

In [3]:
def bar2rel(df,tolerance):
    prev_vals = None
    fix_open_price  = True
    o, h, l, c, v = [], [], [], [], []
    count_out = 0
    count_filter = 0
    count_fixed = 0
    for row in df.itertuples():
        val = (row._3,row._4,row._5,row._6,row._7)
        po, ph, pl,pc,pv = val
        if fix_open_price and prev_vals is not None:
            ppo, pph, ppl, ppc, ppv = prev_vals
            if abs(po - ppc) > 1e-8:
                count_fixed += 1
                po = ppc
                pl = min(pl, po)
                ph = max(ph, po)
                count_out += 1
        o.append(po)
        c.append(pc)
        h.append(ph)
        l.append(pl)
        v.append(pv)
        prev_vals = val
    prices=Prices(open=np.array(o, dtype=np.float32),
                  high=np.array(h, dtype=np.float32),
                  low=np.array(l, dtype=np.float32),
                  close=np.array(c, dtype=np.float32),
                  volume=np.array(v, dtype=np.float32))
    return prices_to_relative(prices)

def prices_to_relative(prices):
    """
    Convert prices to relative in respect to open price
    :param ochl: tuple with open, close, high, low
    :return: tuple with open, rel_close, rel_high, rel_low
    """
    assert isinstance(prices, Prices)
    rh = (prices.high - prices.open) / prices.open
    rl = (prices.low - prices.open) / prices.open
    rc = (prices.close - prices.open) / prices.open
    return Prices(open=prices.open, high=rh, low=rl, close=rc, volume=prices.volume)

def preprocess(path):
    df = pd.read_csv(os.path.abspath(train_path))

    index = ['<OPEN>', "<HIGH>", "<LOW>","<CLOSE>","<VOL>"]
    df[index] = df[index].astype(float)
    df_normalized = (df - df.min()) / (df.max() - df.min())
    # Define the tolerance value
    tolerance = 1e-8

    # Apply the lambda function to check if each value is within the tolerance of the first value
    df_normalized.applymap(lambda v: abs(v - df_normalized.iloc[0]) < tolerance)
    return bar2rel(df_normalized,tolerance)

In [ ]:
import gym
import gym.spaces
from gym.utils import seeding
from gym.envs.registration import EnvSpec
import enum
import numpy as np


DEFAULT_BARS_COUNT = 10
DEFAULT_COMMISSION_PERC = 0.1



class Actions(enum.Enum):
    Skip = 0
    Buy = 1
    Close = 2

class State:
    def __init__(self, bars_count, commission_perc,
                 reset_on_close, reward_on_close=True,
                 volumes=True):
        assert isinstance(bars_count, int)
        assert bars_count > 0
        assert isinstance(commission_perc, float)
        assert commission_perc >= 0.0
        assert isinstance(reset_on_close, bool)
        assert isinstance(reward_on_close, bool)
        self.bars_count = bars_count
        self.commission_perc = commission_perc
        self.reset_on_close = reset_on_close
        self.reward_on_close = reward_on_close
        self.volumes = volumes

    def reset(self, prices, offset):
        assert isinstance(prices, Prices)
        assert offset >= self.bars_count-1
        self.have_position = False
        self.open_price = 0.0
        self._prices = prices
        self._offset = offset

    @property
    def shape(self):
        # [h, l, c] * bars + position_flag + rel_profit
        if self.volumes:
            return 4 * self.bars_count + 1 + 1,
        else:
            return 3*self.bars_count + 1 + 1,

    def encode(self):
        """
        Convert current state into numpy array.
        """
        res = np.ndarray(shape=self.shape, dtype=np.float32)
        shift = 0
        for bar_idx in range(-self.bars_count+1, 1):
            ofs = self._offset + bar_idx
            
            res[shift] = self._prices.high[ofs]
            shift += 1
            res[shift] = self._prices.low[ofs]
            shift += 1
            res[shift] = self._prices.close[ofs]
            shift += 1
            if self.volumes:
                res[shift] = self._prices.volume[ofs]
                shift += 1
            # print(f"""state off set ofs {ofs}\n and shape res as batch from offset {res.shape} \n 
            #         state_offset {self._offset} \n bar_idx {bar_idx} \n shift {shift} \n res shift {res}""")
        res[shift] = float(self.have_position)
        shift += 1
        if not self.have_position:
            res[shift] = 0.0
        else:
            res[shift] = self._cur_close() / self.open_price - 1.0
        # print(f"Final res shape {res.shape} shift {shift}")
        return res

    def _cur_close(self):
        """
        Calculate real close price for the current bar
        Real Close Price = Open Price * (1.0 + Relative Close Price)
                 = 100 * (1.0 + 0.05)
                 = 100 * 1.05
                 = 105

        """
        open = self._prices.open[self._offset]
        rel_close = self._prices.close[self._offset]
        return open * (1.0 + rel_close)

    def step(self, action):
        """
        Perform one step in our price, adjust offset, check for the end of prices
        and handle position change
        :param action:
        :return: reward, done
        """
        assert isinstance(action, Actions)
        reward = 0.0
        done = False
        close = self._cur_close()
        if action == Actions.Buy and not self.have_position:
            self.have_position = True
            self.open_price = close
            reward -= self.commission_perc
        elif action == Actions.Close and self.have_position:
            reward -= self.commission_perc
            done |= self.reset_on_close
            if self.reward_on_close:
                reward += 100.0 * (close / self.open_price - 1.0)
            self.have_position = False
            self.open_price = 0.0

        self._offset += 1
        prev_close = close
        close = self._cur_close()
        done |= self._offset >= self._prices.close.shape[0]-1

        if self.have_position and not self.reward_on_close:
            reward += 100.0 * (close / prev_close - 1.0)

        return reward, done


class StocksEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    #spec = EnvSpec("StocksEnv-v0",entry_point=libs.envoiran.StocksEnv)

    def __init__(self, prices: Prices, bars_count=DEFAULT_BARS_COUNT,
                 commission=DEFAULT_COMMISSION_PERC,
                 reset_on_close=True, state_1d=False,
                 random_ofs_on_reset=True, reward_on_close=False,
                 volumes=False):
        self._prices = prices
        self._state = State(
            bars_count, commission, reset_on_close,
            reward_on_close=reward_on_close, volumes=volumes)
        self.action_space = gym.spaces.Discrete(n=len(Actions))
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf,
            shape=self._state.shape, dtype=np.float32)
        self.random_ofs_on_reset = random_ofs_on_reset
        
        #self.seed()
    def seed(self, seed=None):
        self.np_random, seed1 = seeding.np_random(seed)
        seed2 = seeding.hash_seed(seed1 + 1) % 2 ** 31
        return [seed1, seed2]
    
    def reset(self):
        self._instrument = self.np_random.choice(
            list(self._prices._fields))
        if self._instrument is "open":
            prices = self._prices.open
        if self._instrument is "close":
            prices = self._prices.close
        if self._instrument is "high":
            prices = self._prices.high
        if self._instrument is "low":
            prices = self._prices.low
        else:
            prices = self._prices.volume
        bars = self._state.bars_count
        if self.random_ofs_on_reset:
            offset = self.np_random.choice(
                prices.shape[0]-bars*10) + bars
        else:
            offset = bars
        # print(self._prices.low[offset],offset)
        
        # return P, offset
        self._state.reset(self._prices, offset)
        return self._state.encode()
    def step(self, action_idx):
        
        action = Actions(action_idx)
        reward, done = self._state.step(action)
        obs = self._state.encode()
        info = {

            
                "instrument": self._instrument,
                "offset": self._state._offset
                }
        return obs, reward, done,info #observation, reward, terminated, truncated, info

    


In [ ]:
rp=preprocess(train_path)


In [6]:
env  = StocksEnv(rp, bars_count=10,
                 commission=0.1,
                 reset_on_close=True, state_1d=False,
                 random_ofs_on_reset=True, reward_on_close=False,
                 volumes=True)

All done for env .
lets setup model to get q value from our observation.

In [8]:
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
# Policy Gradient

In [7]:
import gym
import ptan
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

GAMMA = 0.99
LEARNING_RATE = 0.01
EPISODES_TO_TRAIN = 4


class PGN(nn.Module):
    def __init__(self, input_size, n_actions):
        super(PGN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [9]:
def calc_qvals(rewards):
    res = []
    sum_r = 0.0
    for r in reversed(rewards):
        sum_r *= GAMMA
        sum_r += r
        res.append(sum_r)
    return list(reversed(res))

In [10]:
train_path = "D:\Practical_Reinforcement_Learning\chapter8\data\ch08-small-quotes\YNDX_150101_151231.csv"
val_path = "D:\Practical_Reinforcement_Learning\chapter8\data\ch08-small-quotes\YNDX_150101_151231.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


BATCH_SIZE = 32
BARS_COUNT = 10

EPS_START = 1.0
EPS_FINAL = 0.1
EPS_STEPS = 1000000

GAMMA = 0.99

REPLAY_SIZE = 100000
REPLAY_INITIAL = 10000
REWARD_STEPS = 2
LEARNING_RATE = 0.0001
STATES_TO_EVALUATE = 1000


#env = gym.wrappers.TimeLimit(env, max_episode_steps=1000)
# vp = preprocess(val_path)
# env_val = StocksEnv(vp, bars_count=10,
#                  commission=0.1,
#                  reset_on_close=True, state_1d=False,
#                  random_ofs_on_reset=True, reward_on_close=False,
#                  volumes=True)
writer = SummaryWriter(comment="-tradingAgent-reinforce")

net = PGN(env.observation_space.shape[0], env.action_space.n)
print(net)


PGN(
  (net): Sequential(
    (0): Linear(in_features=42, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=3, bias=True)
  )
)


In [11]:
from collections import namedtuple, deque
def _group_list(items, lens):
    """
    Unflat the list of items by lens
    :param items: list of items
    :param lens: list of integers
    :return: list of list of items grouped by lengths
    """
    res = []
    cur_ofs = 0
    for g_len in lens:
        res.append(items[cur_ofs:cur_ofs+g_len])
        cur_ofs += g_len
    return res

# one single experience step
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'done'])


class ProbabilityActionSelector(ptan.actions.ActionSelector):
    """
    Converts probabilities of actions into action by sampling them
    """
    def __call__(self, scores):
        assert isinstance(scores, np.ndarray)
        probs = np.exp(scores - np.max(scores)) / np.sum(np.exp(scores - np.max(scores)))
        actions = []
        print(probs)
        
        for prob in probs:
            if np.any(np.isnan(prob)):
                action = np.random.choice(len(prob), p=np.array([0.83, 0.02,0.15]))
                actions.append(action)
            else:
                action = np.random.choice(len(prob), p=np.array(prob).reshape(-1))
                actions.append(action)

        return np.array(actions)
    

class PolicyAgent(ptan.agent.BaseAgent):
    """
    Policy agent gets action probabilities from the model and samples actions from it
    """
    # TODO: unify code with DQNAgent, as only action selector is differs.
    def __init__(self, model, action_selector=ProbabilityActionSelector(), device="cpu",
                 apply_softmax=False, preprocessor=ptan.agent.default_states_preprocessor):
        self.model = model
        self.action_selector = action_selector
        self.device = device
        self.apply_softmax = apply_softmax
        self.preprocessor = preprocessor

    @torch.no_grad()
    def __call__(self, states, agent_states=None):
        """
        Return actions from given list of states
        :param states: list of states
        :return: list of actions
        """
        if agent_states is None:
            agent_states = [None] * len(states)
        if self.preprocessor is not None:
            states = self.preprocessor(states)
            if torch.is_tensor(states):
                states = states.to(self.device)
        probs_v = self.model(states)
        print(probs_v.size())
        if self.apply_softmax:
            probs_v = F.log_softmax(probs_v, dim=0)
            
        probs = probs_v.data.cpu().numpy()
        
        actions = self.action_selector(probs)
        print(actions)
        return np.array(actions), agent_states
    
class ExperienceSource:
    """
    Simple n-step experience source using single or multiple environments

    Every experience contains n list of Experience entries
    """
    def __init__(self, env, agent, steps_count=2, steps_delta=1, vectorized=False):
        """
        Create simple experience source
        :param env: environment or list of environments to be used
        :param agent: callable to convert batch of states into actions to take
        :param steps_count: count of steps to track for every experience chain
        :param steps_delta: how many steps to do between experience items
        :param vectorized: support of vectorized envs from OpenAI universe
        """
        assert isinstance(env, (gym.Env, list, tuple))
        assert isinstance(agent, ptan.agent.BaseAgent)
        assert isinstance(steps_count, int)
        assert steps_count >= 1
        assert isinstance(vectorized, bool)
        if isinstance(env, (list, tuple)):
            self.pool = env
        else:
            self.pool = [env]
        self.agent = agent
        self.steps_count = steps_count
        self.steps_delta = steps_delta
        self.total_rewards = []
        self.total_steps = []
        self.vectorized = vectorized

    def __iter__(self):
        states, agent_states, histories, cur_rewards, cur_steps = [], [], [], [], []
        env_lens = []
        for env in self.pool:
            obs = env.reset()
            # if the environment is vectorized, all it's output is lists of results.
            # Details are here: https://github.com/openai/universe/blob/master/doc/env_semantics.rst
            if self.vectorized:
                obs_len = len(obs)
                states.extend(obs)
            else:
                obs_len = 1
                states.append(obs)
            env_lens.append(obs_len)

            for _ in range(obs_len):
                histories.append(deque(maxlen=self.steps_count))
                cur_rewards.append(0.0)
                cur_steps.append(0)
                agent_states.append(self.agent.initial_state())

        iter_idx = 0
        while True:
            actions = [None] * len(states)
            states_input = []
            states_indices = []
            for idx, state in enumerate(states):
                if state is None:
                    actions[idx] = self.pool[0].action_space.sample()  # assume that all envs are from the same family
                else:
                    states_input.append(state)
                    states_indices.append(idx)
            if states_input:
                states_actions, new_agent_states = self.agent(states_input, agent_states)
                for idx, action in enumerate(states_actions):
                    g_idx = states_indices[idx]
                    actions[g_idx] = action
                    agent_states[g_idx] = new_agent_states[idx]
            grouped_actions = _group_list(actions, env_lens)

            global_ofs = 0
            for env_idx, (env, action_n) in enumerate(zip(self.pool, grouped_actions)):
                if self.vectorized:
                    next_state_n, r_n, is_done_n, _ = env.step(action_n)
                else:
                    next_state, r, is_done, _ = env.step(action_n[0])
                    next_state_n, r_n, is_done_n = [next_state], [r], [is_done]

                for ofs, (action, next_state, r, is_done) in enumerate(zip(action_n, next_state_n, r_n, is_done_n)):
                    idx = global_ofs + ofs
                    state = states[idx]
                    history = histories[idx]

                    cur_rewards[idx] += r
                    cur_steps[idx] += 1
                    if state is not None:
                        history.append(Experience(state=state, action=action, reward=r, done=is_done))
                    if len(history) == self.steps_count and iter_idx % self.steps_delta == 0:
                        yield tuple(history)
                    states[idx] = next_state
                    if is_done:
                        # in case of very short episode (shorter than our steps count), send gathered history
                        if 0 < len(history) < self.steps_count:
                            yield tuple(history)
                        # generate tail of history
                        while len(history) > 1:
                            history.popleft()
                            yield tuple(history)
                        self.total_rewards.append(cur_rewards[idx])
                        self.total_steps.append(cur_steps[idx])
                        cur_rewards[idx] = 0.0
                        cur_steps[idx] = 0
                        # vectorized envs are reset automatically
                        states[idx] = env.reset() if not self.vectorized else None
                        agent_states[idx] = self.agent.initial_state()
                        history.clear()
                global_ofs += len(action_n)
            iter_idx += 1

In [13]:

agent = PolicyAgent(net, preprocessor=ptan.agent.float32_preprocessor,
                                apply_softmax=True)

#exp_source = ExperienceSource(env, agent, gamma=GAMMA,vectorized=)
exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=GAMMA,vectorized=False)

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)


In [15]:

total_rewards = []
step_idx = 0
done_episodes = 0

batch_episodes = 0
batch_states, batch_actions, batch_qvals = [], [], []
cur_rewards = []


In [ ]:
for step_idx, exp in enumerate(exp_source):
    batch_states.append(exp.state)
    batch_actions.append(int(exp.action))
    cur_rewards.append(exp.reward)

    if exp.last_state is None:
        batch_qvals.extend(calc_qvals(cur_rewards))
        cur_rewards.clear()
        batch_episodes += 1

    # handle new rewards
    new_rewards = exp_source.pop_total_rewards()
    if new_rewards:
        done_episodes += 1
        reward = new_rewards[0]
        total_rewards.append(reward)
        mean_rewards = float(np.mean(total_rewards[-100:]))
        print("%d: reward: %6.2f, mean_100: %6.2f, episodes: %d" % (
            step_idx, reward, mean_rewards, done_episodes))
        writer.add_scalar("reward", reward, step_idx)
        writer.add_scalar("reward_100", mean_rewards, step_idx)
        writer.add_scalar("episodes", done_episodes, step_idx)
        if mean_rewards > 195:
            print("Solved in %d steps and %d episodes!" % (step_idx, done_episodes))
            break

    if batch_episodes < EPISODES_TO_TRAIN:
        continue

    optimizer.zero_grad()
    states_v = torch.FloatTensor(batch_states)
    batch_actions_t = torch.LongTensor(batch_actions)
    batch_qvals_v = torch.FloatTensor(batch_qvals)

    logits_v = net(states_v)
    log_prob_v = F.log_softmax(logits_v, dim=1)
    log_prob_actions_v = batch_qvals_v * log_prob_v[range(len(batch_states)), batch_actions_t]
    loss_v = -log_prob_actions_v.mean()

    loss_v.backward()
    optimizer.step()

    batch_episodes = 0
    batch_states.clear()
    batch_actions.clear()
    batch_qvals.clear()

writer.close()


In [18]:
torch.save(agent.model.state_dict(), 'model_state_dict.pth')


VAlidation


In [20]:
METRICS = (
    'episode_reward',
    'episode_steps',
    'order_profits',
    'order_steps',
)

def train_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = calc_loss(batch=batch,net=net,tgt_net=tgt_net,gamma=GAMMA ** REWARD_STEPS, device=device)
    #print(loss_v)
    loss_v.backward()
    optimizer.step()
    eps_tracker.frame(engine.state.iteration)
    if getattr(engine.state, "eval_states", None) is None:
            eval_states = buffer.sample(STATES_TO_EVALUATE)
            eval_states = [np.array(transition.state, copy=False)
                           for transition in eval_states]
            engine.state.eval_states = np.array(eval_states, copy=False)

    writer.add_scalar("training/loss", loss_v, engine.state.epoch)
    return {
        "loss": loss_v.item(),
        "epsilon": selector.epsilon,
    }

def validation_run(env, net, episodes=100, device="cpu", epsilon=0.02, comission=0.1):
    stats = { metric: [] for metric in METRICS }

    for episode in range(episodes):
        obs = env.reset()

        total_reward = 0.0
        position = None
        position_steps = None
        episode_steps = 0

        while True:
            obs_v = torch.tensor([obs]).to(device)
            out_v = net(obs_v)

            action_idx = out_v.max(dim=1)[1].item()
            if np.random.random() < epsilon:
                action_idx = env.action_space.sample()
            action = Actions(action_idx)
            # closing price at this step
            close_price = env._state._cur_close()

            if action == Actions.Buy and position is None:
                position = close_price
                position_steps = 0
            elif action == Actions.Close and position is not None:
                profit = close_price - position - (close_price + position) * comission / 100
                profit = 100.0 * profit / position
                stats['order_profits'].append(profit)
                stats['order_steps'].append(position_steps)
                position = None
                position_steps = None

            obs, reward, done, _ = env.step(action_idx)
            total_reward += reward
            episode_steps += 1
            if position_steps is not None:
                position_steps += 1
            if done:
                if position is not None:
                    profit = close_price - position - (close_price + position) * comission / 100
                    profit = 100.0 * profit / position
                    stats['order_profits'].append(profit)
                    stats['order_steps'].append(position_steps)
                break

        stats['episode_reward'].append(total_reward)
        stats['episode_steps'].append(episode_steps)

    return { key: np.mean(vals) for key, vals in stats.items() }

In [21]:
trainer = Engine(train_batch)

In [22]:
@trainer.on(Events.COMPLETED | Events.EPOCH_COMPLETED(every=10))
def log_training_results(engine):
    # Check if current epoch is a multiple of 10
    if engine.state.epoch % 10 == 0:
        res=validation_run(env_val, net, episodes=100, device="cpu", epsilon=0.02, comission=0.1)
        #print(f"epoch {engine.state.epoch} \n response : {res}")
        for key, value in res.items():
            writer.add_scalar("Agent Metrics",key, value)



@trainer.on(Events.ITERATION_COMPLETED)
def log_something(engine):
    out_dict = engine.state.output
    for key, value in out_dict.items():
        if value is None:
            value = 0.0
        elif isinstance(value, torch.Tensor):  # Check if value is a tensor
            value = value.item()  # Convert tensor to scalar
        writer.add_scalar(f"Iteration Metrics{engine.state.epoch}/{key}", value, engine.state.iteration)

In [23]:
# Checkpointing
checkpoint_handler = ModelCheckpoint(dirname='saved_models', filename_prefix='checkpoint', n_saved=2, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {'model': net})
trainer.run(batch_generator(buffer, REPLAY_INITIAL, BATCH_SIZE),max_epochs=100)
writer.close()   

C:\Users\sushant\AppData\Local\Temp\ipykernel_12968\266377233.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states_v = torch.tensor(states).to(device)
C:\Users\sushant\AppData\Local\Temp\ipykernel_12968\266377233.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_states_v = torch.tensor(next_states).to(device)


In [ ]:
torch.save(net.state_dict(), 'model_state_dict.pth')
res=validation_run(env_val, net, episodes=100, device="cpu", epsilon=0.02, comission=0.1)
print(res)